In [ ]:
!pip install transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=e010235cc80819263dafbc83c1a3e1233c83d0a1bbdcf7191fce9582ac80daef
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from torch.utils.data import DataLoader
from sentence_transformers import losses, util,models
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers.util import batch_to_device

Mounted at /content/drive


ModuleNotFoundError: ignored

In [ ]:
import pandas as pd
grant = pd.read_csv('/content/drive/MyDrive/innovae-revision/innovae-adavae/data/patent_grant_cleanded_new.csv').sample(frac = 1,random_state = 10).reset_index(drop = True)

#train set split
train_data = grant[grant['set'] == 'train'].reset_index(drop = True)
test_data = grant[grant['set'] == 'val'].reset_index(drop = True)

In [ ]:
def prepare_data_for_paraphrase_mining_evaluator(similar_data):
    sentences_map = {}
    duplicates_list = []

    for idx, pair in enumerate(similar_data):
        text1_id = f"s{2 * idx + 1}"
        text2_id = f"s{2 * idx + 2}"

        sentences_map[text1_id] = pair[0]
        sentences_map[text2_id] = pair[1]

        duplicates_list.append((text1_id, text2_id))

    return sentences_map, duplicates_list

def prepare_data_for_binary_classification_evaluator(similar_data, irrelevant_data):
    sentence1 = []
    sentence2 = []
    labels = []

    # Add similar pairs and their labels
    for t in similar_data:
      sentence1.append(t[0])
      sentence2.append(t[1])
      labels.append(1)

    for i,t in enumerate(similar_data):
      sentence1.append(t[0])
      sentence2.append(irrelevant_data[i])
      labels.append(0)
    return sentence1, sentence2, labels

def callback(score,batch,step):
  print(score,batch,step)
#data_for_similar = [(i,j) for i,j in zip(test_data['primary_claim'],test_data['prior_art_primary_claim'])]
#irelavent_data_similar = [i for i in test_data['3']]

In [ ]:
queries = {}
corpus = {}
relevant_docs = {}

# Prepare data structures
for i, (query, similar_text, dissimilar_text) in enumerate(zip(test_data['primary_claim'],test_data['prior_art_primary_claim'],test_data['0'])):
    query_id = f'query{i+1}'
    similar_doc_id = f'similar_doc{i+1}'
    dissimilar_doc_id = f'dissimilar_doc{i+1}'

    queries[query_id] = query
    corpus[similar_doc_id] = similar_text
    corpus[dissimilar_doc_id] = dissimilar_text

    relevant_docs[query_id] = {similar_doc_id}

######### Read train data  ##########
train_samples = []
for idx,row in train_data.iterrows():
  train_samples.append(InputExample(texts=[row['new_text'], row['prior_text']], label=1))
  train_samples.append(InputExample(texts=[row['prior_text'], row['new_text']], label=1))

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout


#As base model, we use DistilBERT-base that was pre-trained on NLI and STSb data
word_embedding_model = models.Transformer('anferico/bert-for-patents')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout


#As base model, we use DistilBERT-base that was pre-trained on NLI and STSb data
word_embedding_model = models.Transformer('anferico/bert-for-patents')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

auto_model = model._first_module().auto_model

for name, param in auto_model.named_parameters():
  param.requires_grad = False

for name, param in auto_model.named_parameters():
  if any(element in name for element in ['18','19','20','21','22','23','pooler.dense.weight','pooler.dense.bias']):
    param.requires_grad = True

#Training for multiple epochs can be beneficial, as in each epoch a mini-batch is sampled differently
#hence, we get different negatives for each positive
num_epochs = 1

#Increasing the batch size improves the performance for MultipleNegativesRankingLoss. Choose it as large as possible
#I achieved the good results with a batch size of 300-350 (requires about 30 GB of GPU memory)
train_batch_size = 32

model_save_path = 'output/training_MultipleNegativesRankingLoss-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

os.makedirs(model_save_path, exist_ok=True)

# After reading the train_samples, we create a DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size,drop_last = True)
train_loss = losses.MultipleNegativesRankingLoss(model)


################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Questions pair classification,
# Duplicate Questions Mining, and Duplicate Questions Information Retrieval
#evaluators = []

###### Classification ######
# Given (quesiton1, question2), is this a duplicate or not?
# The evaluator will compute the embeddings for both questions and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
#dev_sentences1, dev_sentences2, dev_labels = prepare_data_for_binary_classification_evaluator(data_for_similar, irelavent_data_similar)
#binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, dev_sentences2, dev_labels,show_progress_bar = True)
#evaluators.append(binary_acc_evaluator)

###### Duplicate Questions Mining ######
# Given a large corpus of questions, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
#sentences, paraphrase_pairs = prepare_data_for_paraphrase_mining_evaluator(data_for_paraphrase_mining)
#paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(sentences, paraphrase_pairs, name='dev',show_progress_bar = True)
#evaluators.append(paraphrase_mining_evaluator)

# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
#seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])
train_dataloader.collate_fn = model.smart_batching_collate

#logger.info("Evaluate model without training")
#seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=200,
          output_path=model_save_path,
          callback = callback
          )

model.save("/content/drive/MyDrive/innovae-revision/innovae-adavae/patent_transformer")
model = SentenceTransformer("/content/drive/MyDrive/innovae-revision/innovae-adavae/patent_transformer")

In [ ]:
!pip install chromadb
import chromadb
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import re
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="/content/drive/MyDrive/innovae-revision/innovae-adavae/results/patent_vectorized_database/to/persist/directory" # Optional, defaults to .chromadb/ in the current directory
))

def emb_fn(texts):
  return model.encode(list(texts), convert_to_numpy=True).tolist()

collection = client.create_collection(name="patent_collection_bert", embedding_function=emb_fn,metadata={"hnsw:space": "cosine"})

grant['prior_art_priority_date'] = grant['prior_art_priority_date'].astype('str')
grant['prior_art_subclass_id'] = grant['prior_art_subclass_id'].astype('str')

test_patent = grant[['prior_art_no','prior_art_priority_date','prior_art_subclass_id','prior_text']]
test_patent = test_patent.drop_duplicates(subset = ['prior_art_no'])
test_patent['prior_art_priority_date'] = [str(i) for i in test_patent['prior_art_priority_date']]
docs = test_patent['prior_text'].tolist()
category = pd.DataFrame([i.split('/') for i in test_patent['prior_art_subclass_id']])
category1 = category[0].tolist()
category2 = category[1].tolist()
date = test_patent['prior_art_priority_date'].tolist()
metas = [{'date':d,'cat1':cat1,'cat2':cat2} for d,cat1,cat2 in zip(date, category1, category2)]
ids = [str(i) for i in test_patent['prior_art_no'].tolist()]

recon_dataloader = DataLoader(
      docs,
      batch_size=40,
      pin_memory=True,
      drop_last=False,
      shuffle = False)

embeds = []

for text in tqdm(recon_dataloader):
  embed = model.encode(text)
  embeds.append(embed)

embed = np.concatenate(embeds).tolist()

collection.add(
    documents=docs,
    embeddings = embed,
    ids=ids
)

count = 0
total = 0
for idx,row in test_data.iterrows():
  cat = row['prior_art_subclass_id'].split('/')[0]
  results = collection.query(
    query_texts=[row['new_text']],
    n_results=100)
  total += 1
  if str(row['prior_art_no']) in results['ids'][0]:
    count += 1
  print(total,count)